In [ ]:
!pip install -q transformers datasets evaluate huggingface_hub bitsandbytes rouge_score accelerate gdown

import os
import gc
import torch
import pandas as pd
import gdown
import evaluate
from datasets import Dataset
from google.colab import auth, drive
from google.auth import default
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer,
    DataCollatorForSeq2Seq
)

# Enabled GPU CUDA Memory
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Mount GDrive
drive.mount('/content/drive', force_remount=True)
auth.authenticate_user()
creds, _ = default()

# Download Dataset
file_id = '18cM4Z_GlgHdDruuTXPekmgKk6UU8IBxMjGnuUKZELh0'
output_file = "chatbot_training_data.xlsx"
gdown.download(id=file_id, output=output_file, quiet=False)

# Load Dataset
df = pd.read_excel(output_file)
df = df[['user_input', 'chatbot_response']].dropna().rename(
    columns={'user_input': 'prompt', 'chatbot_response': 'response'}
)
print("Data Loaded:")
print(df.head())

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df).shuffle(seed=42).select(range(50000))
del df
gc.collect()

# Split Dataset
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset, eval_dataset = split_dataset["train"], split_dataset["test"]

# Load Model and Tokenizer
model_name = "google/mt5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# "Size Mismatch" Errors
model.resize_token_embeddings(len(tokenizer))
model.config.use_cache = False
model.gradient_checkpointing_enable()

# Load Evaluation Metric
metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return metric.compute(predictions=decoded_preds, references=decoded_labels)

# Preprocessing Function
MAX_LENGTH = 256
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["prompt"],
        max_length=MAX_LENGTH,
        truncation=True,
    )
    labels = tokenizer(
        examples["response"],
        max_length=MAX_LENGTH,
        truncation=True,
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

# Tokenize Dataset
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=eval_dataset.column_names)
print("Tokenization completed!")

# Training Configuration optimized
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    save_strategy="no",
    #save_total_limit=2,
    num_train_epochs=4,
    gradient_checkpointing=True,
    learning_rate=2e-5,
    weight_decay=0.01,
    #logging_steps=100,
    fp16=torch.cuda.is_available() and not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    optim="adamw_bnb_8bit",  # memory optimizer
    report_to="wandb",
    run_name="MT5_fully_finetuned_chatbot",
)

# Setup Data Collator and Trainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Start Training
print("Starting Training...")
trainer.train()
print("Training Completed!")

# Save Model
print("Saving Model Locally...")
trainer.save_model("./fine_tuned_LLM")
tokenizer.save_pretrained("./fine_tuned_LLM")
print("Model Saved!")

# Upload to Hugging Face
from huggingface_hub import HfApi, create_repo

repo_name = "ArsenKe/MT5_large_finetuned_chatbot"
create_repo(repo_name, exist_ok=True)

# Upload Model to Hugging Face
print("Uploading Model to Hugging Face Hub...")
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)
print(f"Model Uploaded: https://huggingface.co/{repo_name}")


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
  

Downloading...
From (original): https://drive.google.com/uc?id=18cM4Z_GlgHdDruuTXPekmgKk6UU8IBxMjGnuUKZELh0
From (redirected): https://docs.google.com/spreadsheets/d/18cM4Z_GlgHdDruuTXPekmgKk6UU8IBxMjGnuUKZELh0/export?format=xlsx
To: /content/chatbot_training_data.xlsx
1.55MB [00:01, 1.39MB/s]


✅ Data Loaded:
                                              prompt  \
0  User: Could you reserve the Silver Sightseeing...   
1  User: Can I reschedule my E-Rickshaw Tour to a...   
2  User: I need to adjust the number of people fo...   
3  User: Can you recommend a budget-friendly tour...   
4  User: Can I downgrade my Gold Sightseeing Tour...   

                                            response  
0  Chatbot: Of course! The Silver Sightseeing Tou...  
1  Chatbot: No problem! What new date works for y...  
2  Chatbot: Got it! How many people now for the P...  
3  Chatbot: The Silver Sightseeing Tour is perfec...  
4  Chatbot: Yes, we can switch you to a more affo...  


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arsenke (arsenke-fh-tech-wien) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,10.389200
1000,0.651400
1500,0.370100
2000,0.290100
2500,0.229200
3000,0.200900
3500,0.200600
4000,0.160200
4500,0.158500
5000,0.138000


Step,Training Loss
500,10.389200
1000,0.651400
1500,0.370100
2000,0.290100
2500,0.229200
3000,0.200900
3500,0.200600
4000,0.160200
4500,0.158500
5000,0.138000


✅ Training Completed!
💾 Saving Model Locally...
✅ Model Saved!


HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-67e1a6e9-6cd3f33c7179acdd375837ca;29321463-cf5a-49ec-9bd3-fad5e7adf034)

Invalid username or password.

*Test*

In [ ]:
from huggingface_hub import HfApi, create_repo
from huggingface_hub import notebook_login

notebook_login()
repo_name = ".../MT5_large_finetuned_chatbot"

create_repo(repo_name, exist_ok=True)

# Upload Model and Tokenizer
print("Uploading Model to Hugging Face Hub...")
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)
print(f" Model Uploaded: https://huggingface.co/{repo_name}")


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = ".../MT5_large_finetuned_chatbot"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_text = "what tours can i book?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Response: {response}")


In [ ]:
input_text = "what museums i can visit ?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=512)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Response: {response}")

Response: Chatbot: The Platinum Sightseeing Tour showcases Vienna’s museums, including Albertina Museum like St. Stephen’s Cathedral. The cost is 105 euro. Call +43676849696200 for details.Royal E-Cars Tours | Contact: office@royal-ecars.com | Phone: +43676849696200 | Address: Geigergasse 5, 3. Stock, 1050 Wien
